In [1]:
# General libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Datetime library
from datetime import date

# Environment File
import env

# Database path
database = env.database

# Helper functions
import acquire, explore

# Create A Progressive Bar for Loop Operation 
from tqdm.notebook import tqdm

### Data Acquisition

In [120]:
# Print the concise summary of the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7566 entries, 0 to 273
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Symbol          7566 non-null   object 
 1   Name            7566 non-null   object 
 2   Last Sale       7566 non-null   object 
 3   Net Change      7566 non-null   float64
 4   % Change        7548 non-null   object 
 5   Market Cap      7069 non-null   float64
 6   Country         6946 non-null   object 
 7   IPO Year        4285 non-null   float64
 8   Volume          7566 non-null   int64  
 9   Sector          5487 non-null   object 
 10  Industry        5486 non-null   object 
 11  stock_exchange  7566 non-null   object 
dtypes: float64(3), int64(1), object(8)
memory usage: 768.4+ KB


In [121]:
# Sanity check
df["stock_exchange"].value_counts()

NASDAQ    4181
NYSE      3111
AMEX       274
Name: stock_exchange, dtype: int64

**Takeaways**
- Missing values are found in Columns: 
    - `% Change`
    - `Market Cap`
    - `Country`
    - `IOP Year`
    - `Volume`
    - `Sector`
    - `Industry`
- The column names needs to be cleaned.

#### How many sectors in the US stock market and what are they?

In [122]:
df['Sector'].nunique()

12

In [123]:
df['Sector'].value_counts()

Finance                  1419
Health Care              1001
Consumer Services         691
Technology                689
Capital Goods             430
Energy                    220
Public Utilities          205
Basic Industries          205
Consumer Non-Durables     202
Miscellaneous             173
Consumer Durables         133
Transportation            119
Name: Sector, dtype: int64

#### How many industries in the US stock market and what are they?

In [124]:
df['Industry'].nunique()

141

In [125]:
df['Industry'].value_counts().head(10)

Business Services                          592
Major Pharmaceuticals                      531
Major Banks                                329
Real Estate Investment Trusts              246
EDP Services                               212
Computer Software: Prepackaged Software    150
Industrial Machinery/Components            145
Oil & Gas Production                       133
Telecommunications Equipment               107
Medical/Dental Instruments                 101
Name: Industry, dtype: int64

### Select SPY as the Baseline and Compute Its CAGR

In [2]:
%%time

# Specify the start and end date

start_date = "2008-12-31" # year-end adjusted closing price
end_date = "2019-12-31"

# Acquire the adjusted price of SPY from yahoo finance
spy = acquire.acquire_stock_adjclosing('SPY', start_date, end_date)

# Inspect the data
spy

CPU times: user 191 ms, sys: 50.8 ms, total: 242 ms
Wall time: 1.16 s


Date
2008-12-31     70.656677
2009-01-02     72.786407
2009-01-05     72.700264
2009-01-06     73.185722
2009-01-07     70.993355
                 ...    
2019-12-24    314.311646
2019-12-26    315.984802
2019-12-27    315.906494
2019-12-30    314.164856
2019-12-31    314.928040
Name: Adj Close, Length: 2769, dtype: float64

In [3]:
# Compute the begining and ending balances

bb = spy.head(1).values[0]
eb = spy.tail(1).values[0]

# Compute the CAGR of SPY

spy_cagr = (eb/bb)**(1/11) - 1
spy_cagr

0.1455268777592824

### Compute the CAGRs in Batches

In [31]:
%%time

# Create an empty list to hold the stock ticker
symbol=[]

# Create an empty list to hold the CAGR of stocks
cagr=[]

for i in tqdm(['UNH', 'COST', 'NKE', 'VOOG']):
    
    start_date = "1998-12-31"
    end_date = "2019-12-31"
    
    stock = acquire.acquire_stock_adjclosing(i, start_date, end_date)    
    
    if stock.index[0].strftime("%Y-%m-%d") != '1998-12-31':
        continue
    else:
        eb = stock.tail(1).values[0]
        bb = stock.head(1).values[0]
        stock_cagr = (eb/bb)**(1/21) - 1        
        symbol.append(i)
        cagr.append(stock_cagr)
        
        if stock_cagr > 0.15:
            print(f"CAGR of {i}: {stock_cagr: .2%}")
#             print(f"First date: {stock.index[0]}")
        
dic_cagr = {'symbol': symbol, 'CAGR': cagr}
df_cagr = pd.DataFrame.from_dict(dic_cagr)
df_cagr

  0%|          | 0/4 [00:00<?, ?it/s]

CAGR of UNH:  21.89%
CAGR of NKE:  16.55%
CPU times: user 446 ms, sys: 43.7 ms, total: 489 ms
Wall time: 4.33 s


,symbol,CAGR
0,UNH,0.218902
1,COST,0.121654
2,NKE,0.165457


In [38]:
def compute_CAGR(tickers):
    '''
    The function asks the user's input of start and end year and computes the 
    compound annual growth rate (CARG) of interested stocks.
    '''
    # Create an empty list to hold the stock ticker
    symbol=[]
    # Create an empty list to hold the CAGR of stocks
    cagr=[]
    # Enter the start year and end year
    start_year = input()
    end_year = input()
    # Compute the number of years
    year_diff = int(end_year) - int(start_year) + 1
    # Compute the start date and end date
    start_year = str(int(start_year)-1)
    start_date = start_year + "-12-31"
    end_date = end_year + "-12-31"
    # Iterate through the tickers and compute their CAGRs
    for i in tqdm(tickers):
        try:
            stock = acquire.acquire_stock_adjclosing(i, start_date, end_date)
        except (IOError, KeyError):
            continue
        if stock.index[0].strftime("%Y-%m-%d") != start_date:
            continue
        else:
            eb = stock.tail(1).values[0]
            bb = stock.head(1).values[0]
            stock_cagr = (eb/bb)**(1/year_diff) - 1
            symbol.append(i)
            cagr.append(stock_cagr)
            if stock_cagr > 0.20: # Print out the stock of which CAGR is greater than 20%
                print(f"CAGR of {i}: {stock_cagr: .2%}")
    # Save the results
    dic_cagr = {'symbol': symbol, 'CAGR': cagr}
    df_cagr = pd.DataFrame(dic_cagr)
    # Return the dataframe
    return df_cagr

In [39]:
# Test the function
compute_CAGR(['UNH', 'COST', 'NKE', 'VOOG'])

1999
2019


  0%|          | 0/4 [00:00<?, ?it/s]

CAGR of UNH:  21.89%


,symbol,CAGR
0,UNH,0.218902
1,COST,0.121654
2,NKE,0.165457


In [41]:
# Read all the stocks from the US exchanges

df = pd.read_csv(f"{database}usstockexchanges_050721.csv", index_col=0)
df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,stock_exchange
0,A,Agilent Technologies Inc. Common Stock,$133.90,1.5000,1.133%,4.079900e+10,United States,1999.0,1028964,Capital Goods,Electrical Products,NYSE
1,AA,Alcoa Corporation Common Stock,$41.20,1.3100,3.284%,7.693059e+09,NaN,2016.0,4811870,Basic Industries,Metal Fabrications,NYSE
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,$9.825,0.0150,0.153%,1.228248e+09,NaN,2021.0,95765,Finance,Business Services,NYSE
3,AAIC,Arlington Asset Investment Corp Class A (new),$4.11,0.0300,0.735%,1.376077e+08,United States,NaN,414108,Consumer Services,Real Estate Investment Trusts,NYSE
4,AAIC^B,Arlington Asset Investment Corp 7.00%,$24.7057,0.2057,0.84%,NaN,United States,NaN,200,NaN,NaN,NYSE


In [48]:
# Test function compute_CAGR by the first 30 stocks in the list

tickers = df['Symbol'].head(30).to_list()
compute_CAGR(tickers)

1999
2009


  0%|          | 0/30 [00:00<?, ?it/s]

CAGR of ABEV:  28.61%


,symbol,CAGR
0,AA,0.008402
1,AAIC,-0.151093
2,AB,0.071311
3,ABC,0.118394
4,ABEV,0.286126
5,ABM,0.040033
6,ABT,0.045239


In [ ]:
%%time
# Compute the CAGR of stocks that are on the market since 2009-01-01

# Create an empty list to hold the stock ticker
symbol=[]

# Create an empty list to hold the CAGR of stocks
cagr=[]

# Specifiy the start date and end date
start_date = "2009-01-01"
end_date = "2019-12-31"

for i in tqdm(df['Symbol']):
    
    try:
        stock = acquire.acquire_stock_data(i, 'yahoo', start_date, end_date)  
    except (IOError, KeyError):
        continue
    
    if stock.index[0].strftime("%Y-%m-%d") != '2009-01-02':
        continue
    else:
        eb = stock.tail(1).values[0]
        bb = stock.head(1).values[0]
        stock_cagr = (eb/bb)**(1/11) - 1        
        symbol.append(i)
        cagr.append(stock_cagr)
        
        if stock_cagr > 0.142:
            print(f"CAGR of {i}: {stock_cagr: .2%}")
#             print(f"First date: {stock.index[0]}")
        
dic_cagr = {'symbol': symbol, 'CAGR': cagr}
df_cagr = pd.DataFrame.from_dict(dic_cagr)
df_cagr

In [192]:
# Print the consice summary of df_cagr
df_cagr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3017 entries, 0 to 3016
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   symbol  3017 non-null   object 
 1   CAGR    3015 non-null   float64
dtypes: float64(1), object(1)
memory usage: 47.3+ KB


In [199]:
# Inspect df_cagr
df_cagr

,symbol,CAGR
0,A,0.207241
1,AA,-0.017614
2,AAIC,0.162729
3,AAP,0.154051
4,AB,0.113451
...,...,...
3012,WRN,0.136862
3013,WTT,0.108613
3014,WWR,-0.506945
3015,WYY,0.065041


#### What stocks beat SPY on CAGR from 2009-01 to 2019-12?

In [201]:
# Print the number of stocks that beat SPY

mask_beat = (df_cagr.CAGR > 0.142)
mask_beat.sum()

1054

In [205]:
# Rank the stocks on CAGR in a descending order

stocks_beat = df_cagr[mask_beat]
stocks_beat.sort_values(by='CAGR', ascending=False).head(20)

,symbol,CAGR
312,CSR,0.754994
2356,NXST,0.613267
2408,PATK,0.612139
2610,SNBR,0.610762
2409,PAYS,0.594315
1894,EXAS,0.585763
2940,ISDR,0.558587
2698,TREE,0.519458
1845,DXCM,0.488121
2257,MITK,0.483344


**Takeaways**
- Two RuntimeWarnings show up during the computation and the messages are the same: invalid value encountered in double_scalars.
- 3017 stocks have been in the market since 2009-01-01.
- The CAGR field has two missing values.
- 1054 stocks beats SPY on CAGR from 2009-01 to 2019-12. 